In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from keras.layers import Input, Dense, Dropout, LSTM, SimpleRNN
from keras.models import Model, Sequential
import pandas as pd
import os, cv2, itertools
from pylab import rcParams
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import rc
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [106]:
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [ ]:
def plot_loss(history, title):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss: ' + title)
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['Train', 'Validation'], loc='upper right')
    plt.show()
    print('MSE val = ', history.history['val_loss'][-1])

In [ ]:
def plot_y_yhat(x_test, y_test, model, title):
    y_hat = model.predict(x_test)
    error_percent = 100 * (y_hat - y_test) / y_test

    plt.figure(figsize=(20, 10))
    plt.plot(y_test, 'b-', label='True')
    plt.plot(y_hat, 'r--', label='Predicted')
    plt.title('Comparison of Predicted and True Values: ' + title)
    plt.legend(loc='upper right')
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.grid(True)
    plt.show()

    plt.figure(figsize=(20, 10))
    plt.plot(error_percent, 'r-')
    plt.title('Prediction Error Percentage: ' + title)
    plt.ylabel('Error Percentage (%)')
    plt.xlabel('Time')
    plt.grid(True)
    plt.show()

    results = model.evaluate(x_test, y_test)
    print("Test Loss, Test Accuracy:", results)

In [ ]:
def create_sequences(data, sequence_length):
    xs, ys = [], []
    for i in range(len(data) - sequence_length):
        xs.append(data[i:i + sequence_length])
        ys.append(data[i + sequence_length])
    return np.array(xs), np.array(ys)

In [ ]:
data = pd.read_csv(r'C:\Users\...\Downloads\AAPL_hist_data.csv', parse_dates=[0], dayfirst=False) #File Path
data.set_index(data.columns[0], inplace=True)
close_prices = data['Close'].values.reshape(-1, 1)


In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
close_prices_scaled = scaler.fit_transform(close_prices)

In [ ]:
sequence_length = 1  # Adjust based on prior testing or domain knowledge
x, y = create_sequences(close_prices_scaled, sequence_length)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
model = Sequential([
    SimpleRNN(10, input_shape=(sequence_length, 1)),  # Ensure input shape matches the sequence length and features
    Dense(1, activation='linear')])
model.compile(loss='mean_squared_error', optimizer='Adam', metrics=['mae', 'mse'])
model.summary()

Model: "sequential_50"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_50 (SimpleRNN)   (None, 10)                120       
                                                                 
 dense_48 (Dense)            (None, 1)                 11        
                                                                 
Total params: 131
Trainable params: 131
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(x_train, y_train, epochs=100, batch_size=5, validation_split=0.2, verbose=1, shuffle=False)

Epoch 1/100
32/32 [==============================] - 1s 9ms/step - loss: 0.2713 - mae: 0.4617 - mse: 0.2713 - val_loss: 0.1811 - val_mae: 0.3644 - val_mse: 0.1811
Epoch 2/100
32/32 [==============================] - 0s 2ms/step - loss: 0.1073 - mae: 0.2692 - mse: 0.1073 - val_loss: 0.0729 - val_mae: 0.2224 - val_mse: 0.0729
Epoch 3/100
32/32 [==============================] - 0s 2ms/step - loss: 0.0450 - mae: 0.1665 - mse: 0.0450 - val_loss: 0.0398 - val_mae: 0.1577 - val_mse: 0.0398
Epoch 4/100
32/32 [==============================] - 0s 2ms/step - loss: 0.0292 - mae: 0.1343 - mse: 0.0292 - val_loss: 0.0324 - val_mae: 0.1427 - val_mse: 0.0324
Epoch 5/100
32/32 [==============================] - 0s 2ms/step - loss: 0.0257 - mae: 0.1260 - mse: 0.0257 - val_loss: 0.0297 - val_mae: 0.1368 - val_mse: 0.0297
Epoch 6/100
32/32 [==============================] - 0s 2ms/step - loss: 0.0239 - mae: 0.1217 - mse: 0.0239 - val_loss: 0.0274 - val_mae: 0.1312 - val_mse: 0.0274
Epoch 7/100
32/32 [===

In [ ]:
plot_loss(history, 'RNN for Stock Closing Prices')
plot_y_yhat(x_test, y_test, model, 'RNN for Stock Closing Prices')